In [1]:
!bash init.sh

mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘np’: File exists


In [2]:
%cd flowgmm
!pip install -q -e .
!pip install -q timm==0.3.2 torch torchvision
!pip install -q scanpy normflows
!pip install -q pydeseq2
%cd ..

/home/anunay18021/SingleCellClassification/flowgmm
/home/anunay18021/SingleCellClassification


In [3]:
import torch
from torch import nn
from torch.nn.parameter import Parameter
from torch import distributions
import scanpy as sc
import os
from numpy.random import seed
# from tensorflow import set_random_seed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
from pydeseq2.dds import *
dataset_train = 'dataset/Bh.h5ad'
dataset_test = 'dataset/smartseq2.h5ad'
import sys
sys.path.insert(0, '')

from scripts.utils import *
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

In [4]:
def preprocess_deseq(adata_test, n_top_genes = 3000, max_value = 10, get_hvgs=False, scale_and_hvgs = False, calculate_hvg_and_log1p = True):
        """
        INPUT:
        file_path: path to .h5ad containing scRNA-seq
        """
        ## convert to h5ad
        # adata_test = sc.AnnData(genes, labels)
        adata_test.X = adata_test.to_df().to_numpy()
        ## make var names unique
        adata_test.obs_names_make_unique()
        adata_test.var_names_make_unique()
        adata_test = DeseqDataSet(adata = adata_test, design_factors='celltype')
        adata_test.vst()
        ## filter cells with count less than 200

        ## LogNormalise
        if not(scale_and_hvgs):
                return {'data':adata_test}

        if get_hvgs:
                ## Get HVGS
                # 
                if calculate_hvg_and_log1p:
                        sc.pp.log1p(adata_test)
                        sc.pp.highly_variable_genes(adata_test, n_top_genes = n_top_genes)
                adata_test = adata_test[:, adata_test.var.highly_variable]

                ## scale data
                sc.pp.scale(adata_test, max_value=max_value)
                return {'data' : adata_test, 'hvg': adata_test.var.highly_variable}

        ## scale data
        sc.pp.scale(adata_test, max_value=max_value)
        return {'data':adata_test}

In [5]:



adata_train=sc.read(dataset_train)
adata_test=sc.read(dataset_test)

print("Starting preprocessing...")
train_dic = preprocess_deseq(adata_train, get_hvgs = True, scale_and_hvgs = True)
test_dic = preprocess_deseq(adata_test)
print(len(intersection(adata_train.var.index, adata_test.var.index)))
list(adata_train.var.index)[0]

col= [i for i in train_dic['hvg'].index]

train_adata_pp =  train_dic['data']
print(train_dic['hvg'])
test_adata_pp =  test_dic['data'][:, intersection(col, test_dic['data'].var.index)]
train_adata_pp = train_dic['data'][:, intersection(col, train_dic['data'].var.index)]

train_df = train_adata_pp.to_df()
test_df = test_adata_pp.to_df()

## taking common genes
print("Taking common genes...")
final_columns = list(set(train_df.columns).intersection(set(test_df.columns)))
print('Common columns', len(final_columns))
final_columns = [i for i in final_columns if i != 'celltype']
train_df = train_df[final_columns]
test_df = test_df[final_columns]

y_train = train_adata_pp.obs.celltype.to_list()
y_test = test_adata_pp.obs.celltype.to_list()

X_train = train_df.to_numpy()
X_test = test_df.to_numpy()

labels = set(y_train)

mapping = get_mapping(y_train)

y_test = np.array(convert_y_to_mapping(y_test, mapping))

y_train = np.array(convert_y_to_mapping(y_train, mapping))

with open('dataset/np/X_train.pkl', 'wb') as fh:
    pickle.dump(X_train, fh)

with open('dataset/np/X_test.pkl', 'wb') as fh:
    pickle.dump(X_test, fh)

with open('dataset/np/y_test.pkl', 'wb') as fh:
    pickle.dump(y_test, fh)

with open('dataset/np/y_train.pkl', 'wb') as fh:
    pickle.dump(y_train, fh)

Starting preprocessing...
Fitting size factors...


/home/anunay18021/.venv/lib/python3.8/site-packages/pydeseq2/dds.py:259: RuntimeWarning: Every gene contains at least one zero, cannot compute log geometric means. Switching to iterative mode.
  self.fit_size_factors()


Fitting dispersions...
... done in 110.47 seconds.

Fitting MAP dispersions...
... done in 58.44 seconds.



KeyboardInterrupt: 

In [ ]:
labels_y_train = train_adata_pp.obs.celltype.to_list()

In [ ]:
%cd flowgmm

/home/anunay18021/SingleCellClassification/flowgmm


In [ ]:


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.dpi'] = 300

# from flow_ssl.realnvp.realnvp_toy import ToyRealNVP
from flow_ssl.realnvp.realnvp import RealNVPTabular
from flow_ssl.data import make_circles_ssl, make_moons_ssl, make_dataset_from_img, make_dataset_from_npz
from flow_ssl.distributions import SSLGaussMixture
from flow_ssl import FlowLoss

from itertools import chain

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def get_means(means_type, num_means=10, shape=(3, 32, 32), r=1, trainloader=None, device=None, net=None):

    D = np.prod(shape)
    means = torch.zeros((num_means, D)).to(device)

    if means_type == "pixel_const":
        for i in range(num_means):
            means[i, :] = r * (i-4)

    elif means_type == "split_dims":
        mean_portion = D // num_means
        for i in range(num_means):
            means[i, i*mean_portion:(i+1)*mean_portion] = r

    elif means_type == "random":
        for i in range(num_means):
            means[i] = r * torch.randn(D)

    else:
        raise NotImplementedError(means_type)

    return means

In [ ]:
import pickle
with open('../dataset/np/X_train.pkl', 'rb') as fh:
    X_train = pickle.load(fh)

with open('../dataset/np/X_test.pkl', 'rb') as fh:
    X_test = pickle.load(fh)

with open('../dataset/np/y_test.pkl', 'rb') as fh:
    y_test = pickle.load(fh)

with open('../dataset/np/y_train.pkl', 'rb') as fh:
    y_train = pickle.load(fh)

In [ ]:
print(set(list(y_train)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}


In [ ]:
device = torch.device("cuda")

In [ ]:
r = 3.5
means = torch.tensor([[-r, -r], [r, r]])

# r = 3.5
# n_classes = 2
means = get_means('random' , r=0.8*0.7, num_means=len(set(list(y_train))), trainloader=None, shape=(X_train.shape[1]), device=device).to(device)

prior = SSLGaussMixture(means=means, device=device)


flow = RealNVPTabular(num_coupling_layers=7, in_dim=X_train.shape[1], num_layers=3, hidden_dim=1024).to(device)
loss_fn = FlowLoss(prior).to(device)
# get_toy_nvp(D=2, prior=prior, device=None, inner_dim=256, coupling_layers_num=6, inner_layers=2)

In [ ]:
data = X_train
labels = y_train

In [ ]:
torch.set_num_threads(10)

In [ ]:
lr_init = 2e-4
epochs = 1500

n_ul = np.sum(labels == -1)
n_l = np.shape(labels)[0] - n_ul
batch_size = n_l
print_freq = 30

labeled_data = data[labels != -1]
labeled_labels = labels[labels != -1]
unlabeled_data = data[labels == -1]
unlabeled_labels = labels[labels == -1]

optimizer = torch.optim.Adam([p for p in flow.parameters() if p.requires_grad==True], lr=lr_init,
                             weight_decay=1e-2)
step_size = 7500
for t in range(epochs):
    loss_total= []
#     batch_idx = np.random.choice(n_ul, size=batch_size)
#     batch_x, batch_y = unlabeled_data[batch_idx], unlabeled_labels[batch_idx]
    for m in range(batch_size//step_size):
      batch_x = labeled_data[m*step_size: (m + 1)*step_size]
      batch_y = labeled_labels[m*step_size: (m + 1)*step_size]
      batch_x, batch_y = torch.from_numpy(batch_x), torch.from_numpy(batch_y)
      batch_x, batch_y = batch_x.to(device), batch_y.to(device)
      z = flow(batch_x).to(device)
      sldj = flow.logdet().to(device)


      loss = loss_fn(z, sldj, batch_y)

      optimizer.zero_grad()
      loss.backward()#retain_graph=True)
      optimizer.step()

      loss_total.append(loss)

      if t == int(epochs * 0.5) or t == int(epochs * 0.8):
          for p in optimizer.param_groups:
              p["lr"] /= 10
    if t % print_freq == 0:
        print('iter %s:' % t, 'loss = %.3f' % float(sum(loss_total)/len(loss_total)),end="\n")
    if t % (print_freq*10) == 0:
        print()

In [ ]:
torch.save(flow.state_dict(), "flow_baron_deseq.pt")
torch.save(loss_fn.state_dict(), "loss_fn_baron_deseq.pt")

In [ ]:
flow.load_state_dict(torch.load("flow_baron_deseq.pt"))
loss_fn.load_state_dict(torch.load("loss_fn_baron_deseq.pt"))

In [ ]:
data, labels = X_train, y_train
# inv_list = []
# for i in range(data.shape[0]//60000):
#   inv_list.append(flow(torch.from_numpy(data[i*6000:(i+1)*6000]).to(device)).cpu().detach().numpy())

In [ ]:
freq = calculate_freq(y_train)
print(freq)
rev_mapping = {}
for i in mapping:
  rev_mapping[mapping[i]] = i
print(rev_mapping)

In [ ]:
def get_avg_dist(X):
    avg_dist = np.zeros((X.shape[0], ))
    for i in range(X.shape[0]):
        avg_dist[i] = np.mean(np.sqrt((X[i,0] - X[:,0])**2 + (X[i,1] - X[:,1])**2))
    return avg_dist
        
            

In [ ]:
def get_samples_to_generate(freq, generate_upper_limit):
  samples_to_generate = {}
  for i in freq:
    if freq[i] >= generate_upper_limit:
      continue
    else:
      samples_to_generate[i] = generate_upper_limit - freq[i]
  return samples_to_generate

In [ ]:
def generate_data(freq, generate_upper_limit, true_labels, data, x_dist = True):
  samples_to_generate = get_samples_to_generate(freq, generate_upper_limit)
  zs = []
  labels = []
  for i in samples_to_generate:
      z = loss_fn.prior.sample((samples_to_generate[i],), gaussian_id=i).cpu().numpy()
      labels.extend([rev_mapping[i]]*samples_to_generate[i])
      zs.append(z)
  zs = np.concatenate( zs, axis=0 )
  if x_dist:
    zs = flow.inverse(torch.from_numpy(zs).cuda().float()).cpu().detach().numpy()
  y_synthetic = true_labels + labels
  data_synth = np.concatenate((data, zs), axis = 0)
  return data_synth, y_synthetic




In [ ]:
def remove_outliers(X, y):
    dist = get_avg_dist(X)
    mean = dist.mean()
    std = dist.std()
    index = np.where((mean - 3*std <= dist) & (mean + 3*std >= dist))
    X_new = X[index, :][0,:,:]
    Y_new = y[index]
    z_adata_pp = sc.AnnData(X=X_new, obs={"celltype": Y_new})
    z_adata_pp.obsm['X_tsne'] = X_new
    return z_adata_pp

In [ ]:
def plot_tsne_data(X, labels, title):
  z_adata_pp = sc.AnnData(X=X, obs={"celltype": np.array(labels)})
  z_adata_pp.obsm['z_samples'] = X
  sc.tl.tsne(z_adata_pp, use_rep= 'z_samples', random_state=0)
  z_adata_pp = remove_outliers(z_adata_pp.obsm['X_tsne'], z_adata_pp.obs['celltype'].to_numpy())
  with plt.rc_context({'figure.figsize': (5, 5)}):
    sc.pl.tsne(z_adata_pp,
               color='celltype',
               add_outline=True,
               legend_fontsize=14,
               legend_fontoutline=2,
               frameon=False,
               title=title,
               size = 50,
               palette="tab20b",
              )

In [ ]:
generated_data, generated_labels = generate_data(freq, 2000, train_adata_pp.obs.celltype.to_list(), data, True)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq X space visualisation of X upscaled by data generated using NF, 2000 threshold")

In [ ]:
inv = flow(torch.from_numpy(data).to(device)).cpu().detach().numpy()
generated_data, generated_labels = generate_data(freq, 2000, train_adata_pp.obs.celltype.to_list(), inv, False)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq Z space visualisation of X upscaled by data generated using NF, 2000 threshold")

In [ ]:
generated_data, generated_labels = generate_data(freq, 1000, train_adata_pp.obs.celltype.to_list(), data, True)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq X space visualisation of X upscaled by data generated using NF, 1000 threshold")

In [ ]:
inv = flow(torch.from_numpy(data).to(device)).cpu().detach().numpy()
generated_data, generated_labels = generate_data(freq, 1000, train_adata_pp.obs.celltype.to_list(), inv, False)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq Z space visualisation of X upscaled by data generated using NF, 1000 threshold")

In [ ]:
generated_data, generated_labels = generate_data(freq, 500, train_adata_pp.obs.celltype.to_list(), data, True)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq X space visualisation of X upscaled by data generated using NF, 500 threshold")

In [ ]:
inv = flow(torch.from_numpy(data).to(device)).cpu().detach().numpy()
generated_data, generated_labels = generate_data(freq, 500, train_adata_pp.obs.celltype.to_list(), inv, False)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq Z space visualisation of X upscaled by data generated using NF, 500 threshold")

In [ ]:
generated_data, generated_labels = generate_data(freq, 250, train_adata_pp.obs.celltype.to_list(), data, True)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq X space visualisation of X upscaled by data generated using NF, 250 threshold")

In [ ]:
inv = flow(torch.from_numpy(data).to(device)).cpu().detach().numpy()
generated_data, generated_labels = generate_data(freq, 250, train_adata_pp.obs.celltype.to_list(), inv, False)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq Z space visualisation of X upscaled by data generated using NF, 250 threshold")

In [ ]:
generated_data, generated_labels = generate_data(freq, 125, train_adata_pp.obs.celltype.to_list(), data, True)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq X space visualisation of X upscaled by data generated using NF, 125 threshold")

In [ ]:
inv = flow(torch.from_numpy(data).to(device)).cpu().detach().numpy()
generated_data, generated_labels = generate_data(freq, 125, train_adata_pp.obs.celltype.to_list(), inv, False)
plot_tsne_data(generated_data, generated_labels, "TSNE deseq Z space visualisation of X upscaled by data generated using NF, 125 threshold")